<a href="https://colab.research.google.com/github/gcosma/COP509/blob/main/LabSolutions/Lab_Exercise(No_7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial** - Topic Modeling with BERTopic on the Art Reviews Dataset

In this tutorial we will be exploring how to use BERTopic to create topics from the Art reviews dataset. The most frequent use-cases and methods are discussed together with important parameters to keep a look out for.


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
We use the Arts review dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
 !ls "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"


ArtsRatings_5000_test.txt   ArtsReviews_5000_train.txt	Reduced_ArtsRatings_5000.txt
ArtsRatings_5000_train.txt  glove.6B.100d.txt		Reduced_ArtsReviews_5000.txt
ArtsReviews_5000_test.txt   metamorphosis_clean.txt	review_polarity


In [ ]:
data_path = "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"
# !ls "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"

datasets = 'Arts5000' # ArtsAll, Arts5000

if datasets == 'ArtsAll':
  # Arts all
  data_all_reviews = 'ArtsReviews_train.txt'
else:
  # Arts 5000
  data_all_reviews = 'ArtsReviews_5000_train.txt'

# Obtain the reviews in list (docs)
docs = []
f = open(data_path + data_all_reviews)
lines = f.readlines()
for line in lines:
   docs.append(line)
print("Number of reviews:", len(docs))

Number of reviews: 4000


# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model.




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model.


Set the method of reduce dimensionality:

In [ ]:
# Please select from (UMAP, SVD, PCA)
RedDiMethod = 'SVD'

In [ ]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.cluster import KMeans
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic import BERTopic

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
if RedDiMethod == 'SVD':
   RD_model = TruncatedSVD(n_components=50)
elif RedDiMethod == 'PCA':
   RD_model = PCA(n_components='mle')
else:
   RD_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')


# Step 3 - Cluster reduced embeddings
Cluster_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,    # Step 1 - Extract embeddings
  umap_model=RD_model,                # Step 2 - Reduce dimensionality
  hdbscan_model=Cluster_model,        # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,          # Step 5 - Extract topic words
 # diversity=0.5,                      # Step 6 - Diversify topic words
  calculate_probabilities=True
)

topics, probs = topic_model.fit_transform(docs)

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents.

In [ ]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3101,-1_use_product_just_like,"[use, product, just, like, good, great, paper,...",[I love this cute little trimmer!I didn't buy ...
1,0,453,0_machine_sewing_thread_bobbin,"[machine, sewing, thread, bobbin, sew, tension...",[I am a sewist with intermediate sewing skills...
2,1,139,1_scissors_cut_pair_cutting,"[scissors, cut, pair, cutting, shears, blades,...","[I am a very avid sewer. I make hats, purses, ..."
3,2,129,2_paper_ink_pens_stamp,"[paper, ink, pens, stamp, stamps, pen, set, us...",[I began my quest for the perfect colored pape...
4,3,81,3_punch_hole_paper_holes,"[punch, hole, paper, holes, punches, sheets, u...",[All I want a 3-hole punch to do is: 1) to pun...
5,4,35,4_glue_gun_stick_trigger,"[glue, gun, stick, trigger, cement, sticks, us...",[This glue gun worked badly from the first tim...
6,5,30,5_yarn_blanket_bad_baby,"[yarn, blanket, bad, baby, soft, knitting, sec...",[This yarn is soft but that is the best I can ...
7,6,17,6_thank_item_say_product,"[thank, item, say, product, order, cheaper, re...",[After ordering this I found one much cheaper ...
8,7,15,7_book_sibley_birds_field,"[book, sibley, birds, field, bird, golden, gui...",[My husband and I are becoming more interested...


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

[('machine', 0.11910600671954989),
 ('sewing', 0.06568463945296861),
 ('thread', 0.046932892064770616),
 ('bobbin', 0.032197756075776886),
 ('sew', 0.027346039406824205),
 ('tension', 0.026287958500309124),
 ('use', 0.02531130133836808),
 ('brother', 0.025014766744033762),
 ('just', 0.02249764002194085),
 ('stitch', 0.022019989580862827)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs.

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [ ]:
topic_model.topics_[:10]

[-1, -1, 0, -1, -1, -1, -1, -1, -1, -1]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can
be used to understand how confident BERTopic is that certain topics can be found in a document.

To visualize the distributions, we simply call:

In [ ]:
topic_model.visualize_distribution(probs[100], min_probability=0.0001)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=6, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update
the topic representation with new parameters for `c-TF-IDF`:


In [ ]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('machine', 0.04490886633586143),
 ('the', 0.0435706051193915),
 ('it', 0.03629624166861754),
 ('to', 0.03495044671949317),
 ('and', 0.03450595761875932),
 ('this', 0.023776131140616694),
 ('sewing', 0.022562098196829795),
 ('is', 0.021494612109317572),
 ('of', 0.020052724164105477),
 ('for', 0.019869673705756638)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so,
is that you can decide the number of topics after knowing how many are actually created. It is difficult to
predict before training your model how many topics that are in your documents and how many will be extracted.
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
topic_model.reduce_topics(docs, nr_topics=60)

In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

[-1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 2, -1, -1, -1, -1, 2, -1, -1, -1, 0, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 6, -1, -1, -1, 0, 0, -1, -1, 4, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, 0, -1, -1, -1, 4, -1, -1, -1, 7, -1, -1, -1, -1, -1, 0, -1, -1, 2, -1, 3, -1, -1, 0, -1, 2, -1, -1, 0, -1, -1, -1, -1, -1, 5, -1, 1, -1, -1, 5, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, 7, 0, 2, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, 1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, 3, -1, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, 0, -1, 1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, 1, -1, 3, -1, -1, 6, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 5, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 4, 0, -1, -1, -1, 3, -1, -1

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "vehicle". Then, we extract the most similar topic and check the results:

In [ ]:
similar_topics, similarity = topic_model.find_topics("machine", top_n=5); similar_topics

[0, 2, 7, 5, 3]

In [ ]:
topic_model.get_topic(0)

[('machine', 0.04490886633586143),
 ('the', 0.0435706051193915),
 ('it', 0.03629624166861754),
 ('to', 0.03495044671949317),
 ('and', 0.03450595761875932),
 ('this', 0.023776131140616694),
 ('sewing', 0.022562098196829795),
 ('is', 0.021494612109317572),
 ('of', 0.020052724164105477),
 ('for', 0.019869673705756638)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [ ]:
# Save model
topic_model.save("my_model")

2025-02-21 20:07:41,390 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
# Load model
my_model = BERTopic.load("my_model")